In [63]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [64]:
# Initialising the CNN
classifier = Sequential()

In [65]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

In [66]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [67]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
json_string = classifier.to_json()
f=open("model/model.json", "w+")
f.write(json_string)

2742

In [68]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/cnn/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/cnn/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

# classifier.fit_generator(training_set,
#                          steps_per_epoch = 8000,
#                          epochs = 25,
#                          validation_data = test_set,
#                          validation_steps = 2000)

classifier.fit_generator(training_set,
                        steps_per_epoch=250,
                        epochs=25,
                        validation_data=test_set,
                        validation_steps=65)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 66s 262ms/step - loss: 0.6757 - acc: 0.5746 - val_loss: 0.6372 - val_acc: 0.6105
Epoch 2/25
250/250 [==============================] - 76s 304ms/step - loss: 0.6111 - acc: 0.6644 - val_loss: 0.5777 - val_acc: 0.7127
Epoch 3/25
250/250 [==============================] - 74s 297ms/step - loss: 0.5657 - acc: 0.7006 - val_loss: 0.6245 - val_acc: 0.6865
Epoch 4/25
250/250 [==============================] - 64s 256ms/step - loss: 0.5260 - acc: 0.7321 - val_loss: 0.5012 - val_acc: 0.7592
Epoch 5/25
250/250 [==============================] - 63s 253ms/step - loss: 0.5096 - acc: 0.7438 - val_loss: 0.5596 - val_acc: 0.7166
Epoch 6/25
250/250 [==============================] - 55s 220ms/step - loss: 0.4896 - acc: 0.7651 - val_loss: 0.4842 - val_acc: 0.7781
Epoch 7/25
250/250 [==============================] - 55s 220ms/step - loss: 0.4662 - acc: 0.7761 - val_lo

In [73]:
# Part 3 - Making new predictions

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/cnn/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

prediction

'cat'

In [74]:
classifier.save_weights('model/weights.txt')